In [2]:
import zipfile
zip_ref = zipfile.ZipFile("../driving_dataset.zip", 'r')
zip_ref.extractall("../data")
zip_ref.close()

In [5]:
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
import keras

data = pandas.read_csv('../data/driving_dataset/data.txt',header = None,sep=' ' )

In [6]:
data = data.values

In [7]:
x_data = data[:,0]
y_data = np.array([float(i) for i in data[:,-1]])
print(x_data.shape,y_data.shape)

(45406,) (45406,)


In [8]:
l = (int)(x_data.shape[0]*0.8)
xtrain = x_data[:l]
ytrain = y_data[:l]
xtest = x_data[l:]
ytest = y_data[l:]
print(xtrain.shape,xtest.shape,ytrain.shape,ytest.shape)

(36324,) (9082,) (36324,) (9082,)


In [9]:

import random
c = list(zip(xtrain,ytrain))
random.shuffle(c)
xtrain,ytrain = zip(*c)


c = list(zip(xtest,ytest))
random.shuffle(c)
xtest,ytest = zip(*c)

In [10]:
from keras.preprocessing import image

def LoadBatch(batch_size,i,xdata,ydata):
  xd = []
  yd = []
  Pointer = 0;
  if i==len(xdata)//batch_size - 1:
      batch_size = len(xdata)%batch_size
  while Pointer<batch_size:
    img = image.load_img("../data/driving_dataset/"+xdata[(Pointer+i*batch_size)%l],color_mode='rgb',target_size=[200,200])
    img = image.img_to_array(img)/255.0
    xd.append(img)
    yd.append(ydata[(Pointer+i*batch_size)%l]*np.pi/180)
    Pointer = Pointer+1
  return np.array(xd),np.array(yd)

In [11]:
from keras.layers import Conv2D,Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, MaxPool2D
from keras.optimizers import Adam
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import Adam

# from keras.adv

In [12]:
model = Sequential()

model.add(Conv2D(24,(5,5),activation='relu',input_shape=(200,200,3),kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(36,(5,5),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(48,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))

model.add(Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(1,activation='tanh',kernel_regularizer=regularizers.l2(0.01)))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 196, 196, 24)      1824      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 24)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 98, 24)        96        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 94, 94, 36)        21636     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 36)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 47, 47, 36)        144       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 45, 48)        1

In [13]:
model.compile(optimizer = Adam(0.0001),loss = 'mse',metrics=['mae'])

In [ ]:
epochs = 5
batch_size = 128
num = len(xtrain)

c=0
l = len(xtest)

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num//batch_size)):
        xs, ys = LoadBatch(batch_size,i,xtrain,ytrain)
        ys.reshape(-1,1)

        model.fit(xs,ys,verbose=0)

        if i%10 == 0:
            score_train = model.evaluate(xs,ys,verbose=0)
            xs1, ys1 = LoadBatch(batch_size,c,xtest,ytest)
            c+=1
            ys1.reshape(-1,1)
            score_test = model.evaluate(xs1,ys1,verbose=0)
            print("Epoch:%d Step:%d Train loss:%f Train mae:%f Test loss:%f Test mae:%f\n"%(epoch+1,i,score_train[0],score_train[1],score_test[0],score_test[1]))

        if i%100 == 0:
            model.save_weights("gkl7.h5")


Epoch:1 Step:0 Train loss:5.650350 Train mae:0.262177 Test loss:5.665772 Test mae:0.237859

Epoch:1 Step:10 Train loss:5.816091 Train mae:0.376898 Test loss:5.634776 Test mae:0.260243

Epoch:1 Step:20 Train loss:5.690842 Train mae:0.455791 Test loss:5.635510 Test mae:0.397052

Epoch:1 Step:30 Train loss:5.719461 Train mae:0.496644 Test loss:5.718343 Test mae:0.430362

Epoch:1 Step:40 Train loss:5.430764 Train mae:0.347173 Test loss:5.409655 Test mae:0.346206

Epoch:1 Step:50 Train loss:5.503658 Train mae:0.318847 Test loss:5.254816 Test mae:0.226629

Epoch:1 Step:60 Train loss:5.376624 Train mae:0.307958 Test loss:5.471498 Test mae:0.294648

Epoch:1 Step:70 Train loss:5.575811 Train mae:0.599543 Test loss:5.614486 Test mae:0.602983

Epoch:1 Step:80 Train loss:5.378864 Train mae:0.498136 Test loss:5.504138 Test mae:0.551051

Epoch:1 Step:90 Train loss:5.363987 Train mae:0.487525 Test loss:5.420603 Test mae:0.519795

Epoch:1 Step:100 Train loss:5.151326 Train mae:0.429801 Test loss:5.468

Epoch:4 Step:10 Train loss:1.002444 Train mae:0.154005 Test loss:1.224704 Test mae:0.283867

Epoch:4 Step:20 Train loss:0.938047 Train mae:0.109551 Test loss:0.983625 Test mae:0.195473

Epoch:4 Step:30 Train loss:0.910158 Train mae:0.148328 Test loss:1.136324 Test mae:0.276964

Epoch:4 Step:40 Train loss:0.807517 Train mae:0.075512 Test loss:0.979436 Test mae:0.244969

Epoch:4 Step:50 Train loss:0.886995 Train mae:0.117724 Test loss:1.015953 Test mae:0.243193

Epoch:4 Step:60 Train loss:0.813731 Train mae:0.112907 Test loss:0.797309 Test mae:0.167146

Epoch:4 Step:70 Train loss:0.843970 Train mae:0.114206 Test loss:0.954856 Test mae:0.236306

Epoch:4 Step:80 Train loss:0.745019 Train mae:0.112287 Test loss:0.864307 Test mae:0.212502

Epoch:4 Step:90 Train loss:0.830308 Train mae:0.125036 Test loss:1.047727 Test mae:0.313098

Epoch:4 Step:100 Train loss:0.732140 Train mae:0.100632 Test loss:0.739348 Test mae:0.194541

Epoch:4 Step:110 Train loss:0.813011 Train mae:0.125341 Test loss:0.9